In [1]:
import os
import sys
from pathlib import Path


PROJECT_ROOT = Path("..").resolve()
sys.path.append(str(PROJECT_ROOT))

PROJECT_ROOT

from src.nlp.summarizer.pipeline import run_summarization_pipeline
from src.nlp.partitioner.pipeline import run_nlp_partition_pipeline
from src.nlp.io import load_transcript_json
from src.nlp.summarizer.summarizer import SummarizerQWEN



C:\Users\chris\miniforge3\envs\segmenta\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
json_path = PROJECT_ROOT / "transcripts" / "sample.json"
segments = load_transcript_json(json_path)

print("Loaded segments:", len(segments))
segments[:3]

Loaded segments: 539


[{'start': 15.18, 'end': 16.86, 'text': "Right, let's get started."},
 {'start': 16.86,
  'end': 25.18,
  'text': 'So when I covered last week or last session was how to create a CV using'},
 {'start': 25.18, 'end': 26.02, 'text': 'Quarto.'}]

In [3]:
results = run_nlp_partition_pipeline(
    whisper_segments=segments,
    max_duration=60,
    max_gap=2.0,
)

topics = results["topics"]

In [4]:
# ------------------------------------
# Run HF summarizer
# ------------------------------------
chapters = run_summarization_pipeline(topics)

print(f"Generated {len(chapters)} chapter summaries.\n")

# ------------------------------------
# Pretty-print each chapter
# ------------------------------------
for chap in chapters:
    print("=" * 100)
    print(f"📌 Topic {chap['topic_id'] + 1}")
    print(f"⏱  {chap['start']} → {chap['end']}")
    
    print(f"\n🎬 Title:\n{chap['title']}")
    print(f"\n📝 Summary:\n{chap['summary']}")
    
    print("\n🔹 Bullet Points:")
    for b in chap["bullets"]:
        print(f" - {b}")
    
    print("\n🔑 Keywords:")
    print(", ".join(chap["keywords"]))
    
    print("\n")


Generated 8 chapter summaries.

📌 Topic 1
⏱  15.18 → 532.65

🎬 Title:
Creating a Personal Website Using Quarto

📝 Summary:
In this lesson, we learned how to create a personal website using Quarto templates. We covered setting up the website on GitHub, configuring the YAML file for customization, and rendering the website locally. Key steps included initializing the project, customizing the YAML file, and running Quarto renders to generate the final HTML files.

🔹 Bullet Points:
 - Setting up the website on GitHub
 - Customizing the YAML file for styling
 - Rendering the website locally

🔑 Keywords:



📌 Topic 2
⏱  532.65 → 808.55

🎬 Title:
Website Navigation and Reference Links

📝 Summary:
This tutorial demonstrates how to create a basic website using QMD, including specifying the structure, adding a navigation bar, changing the layout, and managing references within documents.

🔹 Bullet Points:
 - Specify the website structure and add a navigation bar.
 - Remove the navigation bar and

In [5]:
len(topics[0]["text"].split())

1319

In [6]:
len(topics[0]["text"].split())
summarizer = SummarizerQWEN()

raw = summarizer.summarize_topic(topics[0]["text"])

print(raw)


{'title': 'Creating a Personal Website Using Quarto', 'summary': 'In this session, we learned how to create a personal website using Quarto templates and GitHub hosting. We discussed setting up the project, configuring the YAML file, and rendering the website locally before deploying it to GitHub Pages.', 'bullets': ['Learn to create a personal website using Quarto templates.', 'Understand the importance of GitHub hosting for website deployment.', 'Customize the website structure using YAML front matter.', 'Render the website locally before deploying it to GitHub Pages.'], 'keywords': []}
